# Production history cleaner

This notebook aims at cleaning the data published by RTE over the last 12 years. Two anomalies are targeted:
- RTE data is sometimes missing for the whole system. Thoses hours will be added with NaN values;
- RTE data is sometimes missing for a specific unit and recorded as "*". Those cells will be replaced by NaN to allow analysis without problems.

In [29]:
# Modules import
import math

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import warnings

#### Functions

In [30]:
def target_cleaner(df, target="*", replacement_value=np.nan):
    """Return df free of target values, replaced by a specified replacement caracter."""
    global_counter = 0                                                  # Counter of abdnormal values

    for column in df.columns:
        global_counter += (df[column] == target).sum()           # Counting the abnormal values by column

    df_clean = df.replace(target, replacement_value)      # Replacing target values by the replacement one
    
    print(global_counter, "cells has been targeted and cleaned")

    return df_clean

######################################################################################################################################
def dates_processing(df):
    """Return completed df with continuous dates converted into Pandas datetime format."""

    print("Warning: expect 30 seconds execution time")
    df_clean_date_without_nan = df["DATE"].replace(np.nan, "")          # Replacing NaN values by empty string

    # Split the dates onto the hyphen between hours in the original data, take only the part before for conversion into datetime data
    date_df = pd.to_datetime(df_clean_date_without_nan.str.split('-').str[0], errors="coerce", dayfirst=True)
    print("Dates extracted")

    global_df_clean2 = df.rename(columns={"DATE":"OldDATE"})

    df_with_dates_formatted = pd.concat([date_df, global_df_clean2], axis=1)
    df_with_dates_formatted.pop("Unnamed: 0")
    df_with_dates_formatted.pop("OldDATE")

    print("Dates replaced")

    first_date = date_df[2]
    last_date = date_df[date_df.shape[0]-1]

    real_timedelta_hours = pd.Timedelta(last_date - first_date).total_seconds()/3600 + 1
    real_datehours = pd.Series(pd.date_range(first_date, freq="h", periods=real_timedelta_hours))

    i = 2                                                               # First effective index of dates (0 = Technology, 1 = Capacity (MW))
    j = 0                                                               # Index to browse the real_datehours DataFrame
    dates_indexes = [[0]]                                               # List with indexes to split and reconcatenate after filling temporal blanks
    dates_to_add = []                                                   # List with the dates to add between truncs of df_dates
    loop_lock_flag = True

    while i < (date_df.shape[0]-1):                                     # While we remain in the date_df bounds
        if date_df[i] == real_datehours[j]:
            # If the current date match with its countepart
            loop_lock_flag = True                                       # We're locked in, raise flag
            i += 1                                                      # Increase both selectors
            j += 1
        elif date_df[i] > real_datehours[j]:
            # If the current date jumped into the future
            #print(date_df[i] - real_datehours[j])
            j += 1                                                      # Increase real_datehours selector only
            if loop_lock_flag == True:
                # If and only if we just unlocked...
                dates_to_add.append([real_datehours[j-1]])              # The first incorrect date starts a new period to add
                dates_indexes[-1].append(i-1)                           # The last correct date_df index ends the correct period
                dates_indexes.append([i])                               # The current correct date_df index will start the next correct period
            else:
                # If it a missing date in a row of blanks
                dates_to_add[-1].append(real_datehours[j-1])            # Add it to the list of dates to be added
            #end if
            loop_lock_flag = False                                      # Lower the lock flag
        else:
            # Impossible case that would suggest a wrong parsing somewhere or a wrong date sort in original DataFrame (unlikely)
            break
        #end if
        
    dates_indexes[-1].append(date_df.shape[0]-1)

    print("Segments to truncate and add identified")

    dfs = []                                                       # List that contains the correct parts of date_df

    for i in range(len(dates_indexes)):
        # For each segment of indexes
        index_range = [j for j in range(dates_indexes[i][0],dates_indexes[i][-1]+1)]
        
        # Add the original DataFrame truncature to the list of DataFrames
        dfs.append(df_with_dates_formatted.loc[index_range,df_with_dates_formatted.columns])
        
        if i <= len(dates_to_add) - 1:
            # If there is a date blank to fill...
            # ...create the data to add, i.e. NaN table of the size of the blank...
            data_to_add = [[np.nan for k in range(len(df_with_dates_formatted.columns))] for j in range(len(dates_to_add[i]))]
            # ...update the first column values to fill the date blank
            for j in range(len(dates_to_add[i])):
                data_to_add[j][0] = pd.to_datetime(dates_to_add[i][j])
            #end for
            
            # Add the filler to the list of DataFrames
            dfs.append(pd.DataFrame(data=data_to_add, columns=df_with_dates_formatted.columns))
        #end if
    #end for

    corrected_date_df = pd.concat(dfs, ignore_index=True)           # Merge the truncatures and the fillers, ignore indexs
    corrected_date_df.at[0,"DATE"] = "Technology"
    corrected_date_df.at[1,"DATE"] = "Capacity (MW)"

    neat_global_df = corrected_date_df.set_index("DATE")

    print("Neat dataframe ready")

    return neat_global_df

######################################################################################################################################
def unit_dict_generator(df):
    """Return unit list as dictionnary.
    
    Input:
    - df : the DataFrame to analyse

    Outputs:
    - unit_dict : the dictionnary containing units data
        - Unit technology;
        - Capacity (MW).
    """
    
    unit_dict = {}

    for column in df.columns:
        # For each unit in the DataFrame, add the corresponding key and values to nuke_dict (not_nuke_dict if option enabled)
        unit_dict.update({column : {
            "Technology" : df[column]["Technology"],
            "Capacity (MW)":float(df[column]["Capacity (MW)"])
            }
        })
    #end for
    
    return unit_dict

######################################################################################################################################
def df_shaper(df, excluded_units_dict={}):
    """Exclude units from the original df and return the corresponding dataframe with data as floats and index as pd.Timestamp."""
    df_1 = df.drop(["Technology", "Capacity (MW)"])
    df_1 = df_1.drop(excluded_units_dict.keys(), axis=1)
    df_1 = df_1.astype("float")
    df_1.index = pd.to_datetime(df_1.index)

    return df_1

######################################################################################################################################
def df_offsetting_tool(df, offset=0):
    """Returns df with offset lines of NaN at the beggining and offset lines dropped at the end"""
    # Drop the last row of current_df
    if offset >= 0:
        offset_df = df.drop(df.index[-offset:], axis=0)
    else:
        offset_df = df.drop(df.index[:-offset], axis=0)
    #end if
    
    # Create the new first row of offset_df
    nan_df = pd.DataFrame([[np.nan for i in range(len(df.columns))] for j in range(abs(offset))], columns=df.columns)
    
    if offset >= 0:
        offset_df = pd.concat([nan_df, offset_df], ignore_index=True)
    else:
        offset_df = pd.concat([offset_df, nan_df], ignore_index=True)
    #end if

    offset_df.index = df.index

    return offset_df

######################################################################################################################################
def relative_dataframe(df, unit_dict):
    """Return the normalized dataframe in % of nominal power of each unit."""
    df_shaped = df_shaper(df)
    
    df_relative = df_shaped.copy()

    for column in df.columns:
        df_relative[column] = df_relative[column]/unit_dict[column]["Capacity (MW)"]
    
    return df_relative

######################################################################################################################################
def ramps_dataframe(df, unit_dict):
    """Return the dataframe of evolution in % of nominal power of each unit."""  
    df_relative = relative_dataframe(df, unit_dict)
    
    df_relative_offset = df_relative.copy()

    # Drop the last row of df_relative_offset
    df_relative_offset = df_relative_offset.drop([df_relative_offset.index[-1]])

    # Create the new first row of df_relative_offset
    nan_df = pd.DataFrame([[np.nan for i in range(len(df_relative_offset.columns))]], columns=df_relative_offset.columns)
    df_relative_offset = pd.concat([nan_df, df_relative_offset], ignore_index=True)

    df_relative_offset.index = df_relative.index
    
    df_ramps = df_relative - df_relative_offset

    return df_ramps

######################################################################################################################################
def no_outliers_dataframe(df, unit_dict, min_acceptable_value=-0.05, max_acceptable_value=1.10):
    """Return the dataframe free of outliers."""
    relative_df = relative_dataframe(df, unit_dict)

    relative_df_bool = (relative_df >= min_acceptable_value) & (relative_df <= max_acceptable_value)

    True_row = pd.DataFrame([[True for i in range(len(relative_df_bool.columns))]], columns=relative_df_bool.columns)

    enhanced_relative_df_bool = pd.concat([True_row, True_row, relative_df_bool])
    enhanced_relative_df_bool.index = df.index
    
    no_outliers_df = df[enhanced_relative_df_bool]
    
    return no_outliers_df

######################################################################################################################################
def remove_spurious_inliers(df, unit_dict):
    """Return the dataframe free of potentially spurious values."""
    modified_df = df.copy()
    ramps_df = ramps_dataframe(df, unit_dict)
    ramps_filter_df = ramps_df[abs(ramps_df) > 0.25]
    ramps_filter_df_offset = df_offsetting_tool(ramps_filter_df,1)

    ramps_add_df = ramps_filter_df + ramps_filter_df_offset
    ramps_diff_df = ramps_filter_df - ramps_filter_df_offset
    is_pb_ramp = (ramps_add_df == 0) & (ramps_diff_df != 0)

    counter = 0
    for column in ramps_df.columns:
        for index in is_pb_ramp[is_pb_ramp[column] == True].index:
            print(column, index)
            print(modified_df.at[index - pd.to_timedelta("1 h"), column])
            modified_df.at[index - pd.to_timedelta("1 h"), column] = modified_df.at[index - pd.to_timedelta("2 h"), column]
            print(modified_df.at[index - pd.to_timedelta("1 h"), column])
            counter += 1
        #end for
    #end for
    print(f"{counter} potentially spurrious values were ignored")

    return modified_df

#### Global DataFrame loading

In [31]:
filename = "./dataframe_RTE.csv.xz"                                 # Global dataframe compressed into a .csv.xz with all 2011-2023 data

global_df = pd.read_csv(filename, low_memory=False)                 # Datatypes are mixed in the DataFrame, hence it is suggested to disable chunking
print("DataFrame extracted and loaded")

DataFrame extracted and loaded


In [32]:
#print(global_df)                                                   # Printing the DataFrame is optional but may help in some cases

#### Starred cells synchronous identification

In [33]:
global_df_clean = target_cleaner(global_df)

536336 cells has been targeted and cleaned


#### Dates extraction, formatting, and completion : expect 30 seconds execution time

In [34]:
neat_global_df = dates_processing(global_df_clean)

Dates extracted
Dates replaced
Segments to truncate and add identified
Neat dataframe ready


In [35]:
print(neat_global_df)

                    BLENOD 2 BLENOD 3 BLENOD 4 BOUCHAIN 1 CORDEMAIS 4  \
DATE                                                                    
Technology           Charbon  Charbon  Charbon    Charbon     Charbon   
Capacity (MW)          250.0    250.0    250.0      250.0       600.0   
2011-12-13 00:00:00      0.0    200.0    189.5        0.0       489.0   
2011-12-13 01:00:00     -2.0    203.0      195        0.0       490.0   
2011-12-13 02:00:00     -5.5    159.0      139        0.0       481.0   
...                      ...      ...      ...        ...         ...   
2023-12-31 19:00:00      NaN      NaN      NaN        NaN        -1.0   
2023-12-31 20:00:00      NaN      NaN      NaN        NaN        -1.0   
2023-12-31 21:00:00      NaN      NaN      NaN        NaN        -1.0   
2023-12-31 22:00:00      NaN      NaN      NaN        NaN        -1.0   
2023-12-31 23:00:00      NaN      NaN      NaN        NaN        -1.0   

                    CORDEMAIS 5 EMILE HUCHET 4 EMI

#### Outliers identification and removal

In [36]:
unit_dict = unit_dict_generator(neat_global_df)
print(unit_dict)

{'BLENOD 2': {'Technology': 'Charbon', 'Capacity (MW)': 250.0}, 'BLENOD 3': {'Technology': 'Charbon', 'Capacity (MW)': 250.0}, 'BLENOD 4': {'Technology': 'Charbon', 'Capacity (MW)': 250.0}, 'BOUCHAIN 1': {'Technology': 'Charbon', 'Capacity (MW)': 250.0}, 'CORDEMAIS 4': {'Technology': 'Charbon', 'Capacity (MW)': 600.0}, 'CORDEMAIS 5': {'Technology': 'Charbon', 'Capacity (MW)': 600.0}, 'EMILE HUCHET 4': {'Technology': 'Charbon', 'Capacity (MW)': 125.0}, 'EMILE HUCHET 5': {'Technology': 'Charbon', 'Capacity (MW)': 343.0}, 'EMILE HUCHET 6': {'Technology': 'Charbon', 'Capacity (MW)': 618.0}, 'HAVRE 1': {'Technology': 'Charbon', 'Capacity (MW)': 250.0}, 'HAVRE 2': {'Technology': 'Charbon', 'Capacity (MW)': 600.0}, 'HAVRE 4': {'Technology': 'Charbon', 'Capacity (MW)': 125.0}, 'HORNAING': {'Technology': 'Charbon', 'Capacity (MW)': 235.0}, 'LUCY': {'Technology': 'Charbon', 'Capacity (MW)': 245.0}, 'MAXE 1': {'Technology': 'Charbon', 'Capacity (MW)': 250.0}, 'MAXE 2': {'Technology': 'Charbon', '

In [37]:
no_outliers_df = no_outliers_dataframe(neat_global_df, unit_dict)

In [38]:
#print(no_outliers_df)

In [39]:
no_outliers_and_inliers_df = remove_spurious_inliers(no_outliers_df, unit_dict)

BLENOD 3 2012-03-25 03:00:00
76.5
0
BLENOD 4 2012-03-25 03:00:00
75.5
-4
BOUCHAIN 1 2015-03-29 03:00:00
136
0
CORDEMAIS 4 2014-10-26 02:00:00
164
0
CORDEMAIS 5 2012-03-25 03:00:00
342.5
0
EMILE HUCHET 4 2012-03-25 03:00:00
110.0
0.0
EMILE HUCHET 6 2012-10-26 15:00:00
405.5
589.0
EMILE HUCHET 6 2015-03-29 03:00:00
269.0
0.0
MAXE 2 2014-02-19 20:00:00
232.0
163.5
VITRY 3 2015-03-29 03:00:00
173.5
0
VITRY 4 2012-03-25 03:00:00
74.5
0.0
VITRY 4 2015-03-29 03:00:00
178.5
0
ARRIGHI 1 2013-09-16 15:00:00
41
0
ARRIGHI 1 2014-08-28 16:00:00
36
0
ARRIGHI 1 2014-10-13 20:00:00
71
0
ARRIGHI 1 2015-02-25 15:00:00
54.0
0.0
ARRIGHI 1 2015-03-25 09:00:00
35.5
0.0
ARRIGHI 1 2017-05-18 16:00:00
55.5
0.0
ARRIGHI 1 2017-09-01 12:00:00
66.0
0.0
ARRIGHI 1 2018-10-22 15:00:00
40.0
0.0
ARRIGHI 1 2019-03-08 10:00:00
87.0
0.0
ARRIGHI 1 2021-05-12 11:00:00
54.0
0.0
ARRIGHI 1 2022-07-08 00:00:00
92.5
56.5
ARRIGHI 1 2023-10-29 00:00:00
62.0
-2.0
ARRIGHI 1 2023-12-06 11:00:00
128.5
65.5
ARRIGHI 2 2015-09-30 16:00:0

In [40]:
print(no_outliers_df["BUGEY 2"][pd.to_datetime("01/10/2014 22:00", dayfirst=True):pd.to_datetime("02/10/2014 04:00", dayfirst=True)])
print(no_outliers_and_inliers_df["BUGEY 2"][pd.to_datetime("01/10/2014 22:00", dayfirst=True):pd.to_datetime("02/10/2014 04:00", dayfirst=True)])

DATE
2014-10-01 22:00:00      0.0
2014-10-01 23:00:00    877.0
2014-10-02 00:00:00      0.0
2014-10-02 01:00:00      0.0
2014-10-02 02:00:00      0.0
2014-10-02 03:00:00      0.0
2014-10-02 04:00:00      0.0
Name: BUGEY 2, dtype: object
DATE
2014-10-01 22:00:00    0.0
2014-10-01 23:00:00    0.0
2014-10-02 00:00:00    0.0
2014-10-02 01:00:00    0.0
2014-10-02 02:00:00    0.0
2014-10-02 03:00:00    0.0
2014-10-02 04:00:00    0.0
Name: BUGEY 2, dtype: object


### Neat global DataFrame export
This step can prove quite long: expect 1 minute duration for 2011 - 2023 export

In [41]:
no_outliers_and_inliers_df.to_csv("dataframe_RTE_without_outliers.csv.xz")